In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 40
epoch_update = 5
batch_size = 200
learning_rate = 0.01
weight_decay_value = 0.001
rho_value = 0.9

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

class Swish(torch.nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)


class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.network = nn.Sequential( 
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            Swish(),  # Instantiate Swish directly here
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            Swish(),
            nn.MaxPool2d(2, 2),  # output: 64 x 16 x 16

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            Swish(),
            nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
            Swish(),
            nn.MaxPool2d(2, 2),  # output: 128 x 8 x 8

            nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),
            Swish(),
            nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
            Swish(),
            nn.MaxPool2d(2, 2),  # output: 256 x 4 x 4

            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(256*4*4, 1024),
            nn.BatchNorm1d(1024),
            Swish(),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),
            Swish(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        return self.network(x)



model = ConvNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)


for epoch in range(num_epochs):
    for i , (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if (epoch + 1) % epoch_update == 0:
        print(f'Epoch [{epoch + 1} / {num_epochs}], loss {loss.item():.4f}')
        model.eval()
        with torch.no_grad():
            n_correct = 0
            n_samples = 0
            n_class_correct = [0 for i in range(10)]
            n_class_samples = [0 for i in range(10)]
            print('=====TRAINING=====')
            for images, labels in train_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
        
                _, predicted = torch.max(outputs, 1)
                n_samples += labels.size(0)
                n_correct += (predicted == labels).sum().item()
        
                for i in range(len(labels)):
                    label = labels[i]
                    pred = predicted[i]
                    if (label == pred):
                        n_class_correct[label] += 1
                    n_class_samples[label] += 1
            acc = 100.0 * n_correct / n_samples
            print(f'Accuracy of the network: {acc} %')
            for i in range(10):
                acc = 100.0 * n_class_correct[i] / n_class_samples[i]
                print(f'Accuracy of {classes[i]}: {acc} %')
            print('=====TESTING=====')
            n_correct = 0
            n_samples = 0
            n_class_correct = [0 for i in range(10)]
            n_class_samples = [0 for i in range(10)]
            for images, labels in test_loader:
                images = images.to(device)
                labels = labels.to(device)
                outputs = model(images)
        
                _, predicted = torch.max(outputs, 1)
                n_samples += labels.size(0)
                n_correct += (predicted == labels).sum().item()
        
                for i in range(len(labels)):
                    label = labels[i]
                    pred = predicted[i]
                    if (label == pred):
                        n_class_correct[label] += 1
                    n_class_samples[label] += 1
            acc = 100.0 * n_correct / n_samples
            print(f'Accuracy of the network: {acc} %')
            for i in range(10):
                acc = 100.0 * n_class_correct[i] / n_class_samples[i]
                print(f'Accuracy of {classes[i]}: {acc} %')
            scheduler.step(acc)
            model.train()


Files already downloaded and verified
Files already downloaded and verified
Epoch [5 / 40], loss 0.6964
=====TRAINING=====
Accuracy of the network: 78.072 %
Accuracy of plane: 85.08 %
Accuracy of car: 97.56 %
Accuracy of bird: 68.36 %
Accuracy of cat: 58.84 %
Accuracy of deer: 69.32 %
Accuracy of dog: 71.08 %
Accuracy of frog: 74.6 %
Accuracy of horse: 85.62 %
Accuracy of ship: 88.82 %
Accuracy of truck: 81.44 %
=====TESTING=====
Accuracy of the network: 74.03 %
Accuracy of plane: 83.6 %
Accuracy of car: 95.2 %
Accuracy of bird: 59.8 %
Accuracy of cat: 52.1 %
Accuracy of deer: 65.9 %
Accuracy of dog: 68.4 %
Accuracy of frog: 72.6 %
Accuracy of horse: 80.3 %
Accuracy of ship: 83.9 %
Accuracy of truck: 78.5 %
Epoch [10 / 40], loss 0.3454
=====TRAINING=====
Accuracy of the network: 93.504 %
Accuracy of plane: 94.84 %
Accuracy of car: 98.64 %
Accuracy of bird: 93.46 %
Accuracy of cat: 86.78 %
Accuracy of deer: 94.24 %
Accuracy of dog: 79.66 %
Accuracy of frog: 95.88 %
Accuracy of horse: 96

In [13]:
from time import time

t0=time()
models = []
for i in range(30):
    epoch_update = 40
    print("----------Model", (i+1), "----------")

    model = ConvNet().to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)
    
    
    for epoch in range(num_epochs):
        for i , (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)
    
            outputs = model(images)
            loss = criterion(outputs, labels)
    
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        if (epoch + 1) % epoch_update == 0:
            print(f'Epoch [{epoch + 1} / {num_epochs}], loss {loss.item():.4f}')
            model.eval()
            with torch.no_grad():
                n_correct = 0
                n_samples = 0
                n_class_correct = [0 for i in range(10)]
                n_class_samples = [0 for i in range(10)]
                print('=====TRAINING=====')
                for images, labels in train_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
            
                    _, predicted = torch.max(outputs, 1)
                    n_samples += labels.size(0)
                    n_correct += (predicted == labels).sum().item()
            
                    for i in range(len(labels)):
                        label = labels[i]
                        pred = predicted[i]
                        if (label == pred):
                            n_class_correct[label] += 1
                        n_class_samples[label] += 1
                acc = 100.0 * n_correct / n_samples
                print(f'Accuracy of the network: {acc} %')
                for i in range(10):
                    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
                    print(f'Accuracy of {classes[i]}: {acc} %')
                print('=====TESTING=====')
                n_correct = 0
                n_samples = 0
                n_class_correct = [0 for i in range(10)]
                n_class_samples = [0 for i in range(10)]
                for images, labels in test_loader:
                    images = images.to(device)
                    labels = labels.to(device)
                    outputs = model(images)
            
                    _, predicted = torch.max(outputs, 1)
                    n_samples += labels.size(0)
                    n_correct += (predicted == labels).sum().item()
            
                    for i in range(len(labels)):
                        label = labels[i]
                        pred = predicted[i]
                        if (label == pred):
                            n_class_correct[label] += 1
                        n_class_samples[label] += 1
                acc = 100.0 * n_correct / n_samples
                print(f'Accuracy of the network: {acc} %')
                for i in range(10):
                    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
                    print(f'Accuracy of {classes[i]}: {acc} %')
                scheduler.step(acc)
                model.train()
        models.append(model)
        model.eval()
print(f"Elapsed Time: {(time() - t0):.3f} seconds")

----------Model 1 ----------
Epoch [40 / 40], loss 0.0103
=====TRAINING=====
Accuracy of the network: 99.236 %
Accuracy of plane: 99.32 %
Accuracy of car: 99.66 %
Accuracy of bird: 99.68 %
Accuracy of cat: 97.56 %
Accuracy of deer: 99.54 %
Accuracy of dog: 98.36 %
Accuracy of frog: 99.46 %
Accuracy of horse: 99.54 %
Accuracy of ship: 99.48 %
Accuracy of truck: 99.76 %
=====TESTING=====
Accuracy of the network: 79.15 %
Accuracy of plane: 82.4 %
Accuracy of car: 89.4 %
Accuracy of bird: 73.8 %
Accuracy of cat: 51.9 %
Accuracy of deer: 78.7 %
Accuracy of dog: 68.3 %
Accuracy of frog: 85.6 %
Accuracy of horse: 86.3 %
Accuracy of ship: 86.7 %
Accuracy of truck: 88.4 %
----------Model 2 ----------
Epoch [40 / 40], loss 0.0365
=====TRAINING=====
Accuracy of the network: 98.816 %
Accuracy of plane: 99.82 %
Accuracy of car: 98.94 %
Accuracy of bird: 99.18 %
Accuracy of cat: 99.6 %
Accuracy of deer: 98.18 %
Accuracy of dog: 97.68 %
Accuracy of frog: 98.72 %
Accuracy of horse: 99.0 %
Accuracy of 

In [16]:
from collections import Counter

def majority_vote(predictions, num_classes=10):
    class_votes = np.zeros((len(predictions), 1))
    for i, prediction in enumerate(predictions):
        element_counts = Counter(prediction)
        most_common_element = element_counts.most_common(1)[0][0]
        class_votes[i] = most_common_element
    return class_votes

In [17]:
all_predictions = []

with torch.no_grad():
    for model in models:
        model_predictions = []
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            model_predictions.extend(predicted.cpu().numpy())

        all_predictions.append(model_predictions)
predictions_matrix = np.column_stack(all_predictions)


NameError: name 'test_labels' is not defined

In [21]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)
test_data, test_labels = next(iter(test_loader))
test_labels_list = test_labels.cpu().numpy().tolist()

final_predictions = majority_vote(predictions_matrix)
final_predictions_flat = final_predictions.flatten()
samples = len(final_predictions_flat)  
correct = (final_predictions_flat == test_labels_list).sum() 
accuracy = correct / samples
print(f"Total Accuracy: {100 * accuracy:.4f} %")

Total Accuracy: 86.3700 %
